# Contamination removal using tapioca
    (http://github.com/ncgr/tapioca)
    Removes E.coli, PhiX, and Illumina associated oligos

In [1]:
!source activate py36

In [2]:
import sys

sys.path.append('/home/faske/g/anaconda3/envs/py36/lib/python3.6/site-packages')
sys.path.append("/home/faske/g/ipynb/include_utils")

import ipyparallel as ipp
import os, time
import include_utils as u
import pandas as pd
import numpy as np
import scipy as sp
import numbers
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import matplotlib.colors as mcolors
#import vcf
from sklearn import preprocessing
from subprocess import Popen, PIPE, call, check_output
import seaborn as sns
from IPython.display import FileLink
import urllib.request as urllib2
import dill
import traceback
from pandas import Series, DataFrame
import gzip
import warnings
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)
%config InlineBackend.figure_format = 'retina'
from Bio import SeqIO
#import pysam
from collections import OrderedDict, namedtuple, Counter
import operator
import multiprocessing as mp
import shutil
import tempfile
#from ipyparallel import Client
import scandir
import glob
from Bio.SeqIO.QualityIO import FastqGeneralIterator
import pickle
#import Levenshtein as lv

In [3]:
root = "/home/faske/g/balsam"

In [4]:
cd $root

/working/tfaske/balsam


In [5]:
pwd

'/working/tfaske/balsam'

# Find raw fastq 
    This finds raw fastq files and designates new name assignment for downstream
    once this is changed to fit desired locations, everything should be automatic

In [6]:
fastq_files = []
for OGdir in ['original/']:
    os.chdir('{}/{}'.format(root, OGdir))
    files = !find . -name '*fastq.gz'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        fastq_files.append(x)
fastq_files = sorted(fastq_files)

In [7]:
fastq_files

['/working/tfaske/balsam/original/Library2-BASA_S2_L002_R1_001.fastq.gz']

In [8]:
fastq_dict = dict()
fastq_names = ["BASA1"]
for i in range(0,len(fastq_files)):
    fastq_dict[fastq_names[i]] = fastq_files[i]
fastq_dict

{'BASA1': '/working/tfaske/balsam/original/Library2-BASA_S2_L002_R1_001.fastq.gz'}

# Run tapioca
    once above is changed, this should run without change
    This will make shell scripts for each decontam step separately 

In [15]:
cd $root

/working/tfaske/balsam


In [16]:
!mkdir 'contam'

In [17]:
tap_analysis = '/home/faske/g/src/tapioca/src/./tap_contam_analysis'
tap_contam_dir = '/archive/parchman_lab/rawdata_to_backup/contaminants/'

In [18]:
def write_tap_illum_sh(fastq_dict):
    with open("contam/run_tap_illum.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for name, fastq in fastq_dict.items():
            o.write("""{} --db {}/illumina_oligos --pct 20 {} > {}.readstofilter.ill.txt\n""".format(tap_analysis,tap_contam_dir,fastq,name))

In [19]:
def write_tap_phix_sh(fastq_dict):
    with open("contam/run_tap_phix.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for name, fastq in fastq_dict.items():
            o.write("""{} --db {}/phix174 --pct 80 {} > {}.readstofilter.phix.txt\n""".format(tap_analysis,tap_contam_dir,fastq,name))

In [20]:
def write_tap_ecoli_sh(fastq_dict):
    with open("contam/run_tap_ecoli.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for name, fastq in fastq_dict.items():
            o.write("""{} --db {}/ecoli-k-12 --pct 80 {} > {}.readstofilter.ecoli.txt\n""".format(tap_analysis,tap_contam_dir,fastq,name))

In [21]:
write_tap_illum_sh(fastq_dict),write_tap_phix_sh(fastq_dict),write_tap_ecoli_sh(fastq_dict)

(None, None, None)

# Run contam.sh in terminal

    cd /home/faske/g/balsam/contam
    module load fqutils/0.4.1
    module load bowtie2/2.2.5
    bash run_tap_ecoli.sh &
    disown -h 
    bash run_tap_phix.sh &
    disown -h 
    bash run_tap_illum.sh &
    disown -h 
    

# Removes contaminents and makes new fastq file

In [22]:
cd $root

/working/tfaske/balsam


In [23]:
def write_merge_sh(fastq_dict):
    with open("contam/run_merge.sh", "w") as o:
        o.write("""#!/bin/bash \n""")
        for name, fastq in fastq_dict.items():
            o.write("""cat {} | fqu_cull -r {}.readstofilter.ill.txt {}.readstofilter.phix.txt {}.readstofilter.ecoli.txt > {}.clean.fastq\n""".format(fastq,name,name,name,name))

In [24]:
write_merge_sh(fastq_dict)

# Run contam.sh in terminal
    cd /home/faske/g/dusty/contam
    module load fqutils/0.4.1
    module load bowtie2/2.2.5
    bash run_merge.sh &
    disown -h
    

# This makes a bash script to run other bash scripts
    didnt work on ponderosa. Save for a HPC. 

In [94]:
bash_files = []
for contam_dir in ['contam/']:
    os.chdir('{}/{}'.format(root, contam_dir))
    files = !find . -name '*.sh'
    files = [os.path.abspath(x) for x in files]
    for x in files:
        bash_files.append(x)
bash_files = sorted(bash_files)

In [95]:
bash_files

['/working/tfaske/dusty/contam/run_contam.sh',
 '/working/tfaske/dusty/contam/run_tap_ecoli.sh',
 '/working/tfaske/dusty/contam/run_tap_illum.sh',
 '/working/tfaske/dusty/contam/run_tap_phix.sh']

In [96]:
cd $root 

/working/tfaske/dusty


In [97]:
def write_contam_sh(bash_files):
    with open("contam/run_contam.sh", "w") as o:
        o.write("""#!/bin/bash
#$ -S /bin/bash
#$ -cwd
#$ -o contam.out
#$ -e contam.err\n""")
        for bash in bash_files:
            o.write("""bash {}\n""".format(bash))

In [98]:
write_contam_sh(bash_files)